### Set Up

In [ ]:
!pip install unsloth vllm

### Model

In [ ]:
from unsloth import FastVisionModel
import torch

model, tokenizer = FastVisionModel.from_pretrained(
    "unsloth/Qwen2.5-VL-7B-Instruct-bnb-4bit",
    load_in_4bit = True,
    use_gradient_checkpointing = "unsloth", 
)

In [ ]:
lora_rank = 8

model = FastVisionModel.get_peft_model(
    model,
    finetune_vision_layers     = True, 
    finetune_language_layers   = True, 
    finetune_attention_modules = True,
    finetune_mlp_modules       = True, 

    r = lora_rank,          
    lora_alpha = lora_rank*2, 
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
    use_rslora = False, 
    loftq_config = None, 
    # target_modules = "all-linear", # Optional now! Can specify a list if needed
)

### Dataset

In [ ]:
import json

train_path = "/kaggle/input/vlsp-2025-dataset/train_data/vlsp_2025_train.json"

with open(train_path, "r", encoding="utf-8") as f:
    train = json.load(f)

In [ ]:
from PIL import Image
from datasets import Dataset

system_prompt=(
    "Bạn là một trợ lý pháp lý tiếng Việt chuyên phân tích câu hỏi liên quan đến luật giao thông đường bộ và biển báo. "
    "Luôn trả lời hoàn toàn bằng tiếng Việt.\n\n"
    "Trả lời theo định dạng sau:\n"
    "<think>"
    "[Suy nghĩ, phân tích của bạn]"
    "</think>"
    "[Câu trả lời của bạn]"
)

def create_conversation(input):
    img_path = "/kaggle/input/vlsp-2025-dataset/train_data/train_images/" + input["image_id"] + ".jpg"
    img = Image.open(img_path)

    if input["question_type"] == "Multiple choice":
        if input["answer"] == 40:  # Process edge case
            input["answer"] = "A"

        user_prompt = (
            "Dựa vào bối cảnh bên dưới, hãy phân tích kỹ trước khi trả lời câu hỏi.\n\n"
            "Loại câu hỏi: Trắc nghiệm (kết luận cuối cùng sau khi suy luận là 1 trong 4 lựa chọn: ‘A’, ‘B’, ‘C’, ‘D’. Không được giải thích gì thêm.)\n\n"
            f"Câu hỏi: {input['question']}\n\n"
            "4 lựa chọn:\n"
            f"A: {input['choices']['A']}\n"
            f"B: {input['choices']['B']}\n"
            f"C: {input['choices']['C']}\n"
            f"D: {input['choices']['D']}\n\n"
            "Hãy trả lời theo định dạng:\n"
            "<think>…phân tích chi tiết…</think><answer>A/B/C/D</answer>"
        )
    else:
        user_prompt = (
            "Dựa vào bức ảnh, hãy phân tích kỹ trước khi trả lời câu hỏi.\n\n"
            "Loại câu hỏi: Đúng/Sai (kết luận cuối cùng chỉ là một từ: 'Đúng' hoặc 'Sai'. Không được giải thích gì thêm.)\n\n"
            f"Câu hỏi: {input['question']}\n\n"
            "Hãy trả lời theo định dạng:\n"
            "<think>…phân tích chi tiết…</think><answer>Đúng/Sai</answer>"
        )
    return [
        {"role": "system", "content": system_prompt},
        {
            "role": "user",
            "content": [
                {"type": "text",  "text": user_prompt},
                {"type": "image", "image": img}
            ],
        },
        {
            "role": "assistant",
            "content": [
                {"type": "text", "text": input["answer_think"]}
            ],
        },
    ]

def convert_input_to_grpo(input):
    conversation = create_conversation(input)
    rendered_prompt = tokenizer.apply_chat_template(
        conversation,
        tokenize=False,
        add_generation_prompt=True,
    )
    return {
        "prompt": rendered_prompt,
        "answer": str(input["answer"]),
    }

train_grpo = [convert_input_to_grpo(x) for x in train]
train_ds = Dataset.from_list(train_grpo)

### Reward

In [ ]:
import re
from typing import Dict, Any

_ANSWER_TAG_RE = re.compile(r"<answer>\s*([\s\S]+?)\s*</answer>", flags=re.IGNORECASE)

def _extract_answer(text: str) -> str:
    if not text:
        return ""
    m = _ANSWER_TAG_RE.search(text)
    if m:
        return m.group(1).strip()
    return text.strip().splitlines()[-1].strip()

def reward_function(prompts, completions, **kwargs):
    rewards = []
    
    for prompt, pred_text in zip(prompts, completions):
        score = 0.0

        # Has good formatting
        w_formatting = 1
        
        has_think = "<think>" in pred_text and "</think>" in pred_text
        has_answer = "<answer>" in pred_text and "</answer>" in pred_text
        score += (w_formatting / 2) if has_think else 0
        score += (w_formatting / 2) if has_answer else 0

        # Contain Vietnamese characters
        w_vietnamese = 0.5
        
        vietnamese_chars = "ăâđêôơưáàảãạéèẻẽẹóòỏõọúùủũụíìỉĩị"
        vn_count = sum(c in vietnamese_chars for c in pred_text.lower())
        if vn_count >= 10:
            score += w_vietnamese

        # Reasoning quality
        w_reasoning = 3.5
        
        if has_think:
            think_content = pred_text.split("<think>")[1].split("</think>")[0].strip()
            
            # Adequate length — encourage more developed reasoning
            length_bonus = min(len(think_content.split()) / 100, 1.0) 
            score += w_reasoning * 0.3 * length_bonus

            # Keyword bonus — reward structured analysis reasoning
            reasoning_keywords = [
                "phân tích", "giải thích", "so sánh", "đối chiếu",
                "nguyên nhân", "kết luận", "lý do", "bước", "giả thiết"
            ]
            keyword_hits = sum(kw in think_content.lower() for kw in reasoning_keywords)
            if keyword_hits > 0:
                score += w_reasoning * 0.4 * min(keyword_hits / 3, 1.0)

            # Completeness check — did it consider all answer choices A–D?/ evaluation words (Đúng, Sai)
            multiple_choices_responses = ["a", "b", "c", "d"]
            yes_no_responses = ["đúng", "sai"]

            mc_point = 0.25
            yn_point = 0.5
            
            choices_score = sum(
                mc_point for ch in multiple_choices_responses 
                if re.search(rf"\b{ch}\b", think_content.lower())
            )
            yes_no_score = sum(
                yn_point for word in yes_no_responses 
                if word in think_content.lower()
            )
            score += w_reasoning * max(choices_score, yes_no_score)

        # Correctness
        w_correctness = 5
        
        pred_ans = _extract_answer(pred_text)
        true_ans = kwargs.get("answers", [""] * len(prompts))[len(rewards)]
        if pred_ans.strip().lower() == str(true_ans).strip().lower():
            score += w_correctness
        
        rewards.append(score)
    return rewards

def reward_wrapper(prompts, completions, **kwargs):
    answers = kwargs.get("samples", {}).get("answer", [""] * len(prompts))
    return reward_function(prompts, completions, answers=answers)

### Training

In [ ]:
from trl import GRPOTrainer, GRPOConfig

grpo_config = GRPOConfig(
    output_dir="outputs",
    num_generations=4,     
    learning_rate=2e-4,
    max_completion_length=2048,
    temperature=0.7,
    beta=0.1,
    num_train_epochs = 1,                 
    gradient_accumulation_steps=4,
    fp16=True,
    logging_steps=1,
    save_steps=10,
    save_total_limit = 3,
    seed = 3407,
    report_to=["none"],
)

grpo_config_test = GRPOConfig(
    output_dir="outputs",
    learning_rate=1e-5,
    max_completion_length=2048,
    beta=0.1,
    num_generations=2,
    temperature=0.7,
    logging_steps=1,
    save_steps=5,
    max_steps=10,
    gradient_accumulation_steps=1,
    fp16=True,
    report_to=["none"],
)


trainer = GRPOTrainer(
    model=model,
    tokenizer=tokenizer,
    reward_funcs=reward_wrapper,
    args=grpo_config,
    train_dataset=train_ds,
)

In [ ]:
trainer.train()

In [ ]:
from kaggle_secrets import UserSecretsClient
from huggingface_hub import HfApi

user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("HF_TOKEN")

repo = "venomsnaker/Qwen2.5-VL-7B-Instruct-GRPO"

save_dir = "outputs/final_model"
trainer.save_model(save_dir)  
tokenizer.save_pretrained(save_dir)

model.push_to_hub_merged(repo, tokenizer, save_method = "merged_16bit", token = hf_token)